# Utilizing Artificial Intelligence and Electroencephalography to Assess Expertise on a Simulated Neurosurgical Task
## Sample data available  [HERE](https://www.dropbox.com/s/s86ntxdfayizwcl/data-sample.csv?dl=0)
### Submitted to the Journal of Surgical Education 2021-08-27

In [ ]:
# GENERAL
import os
import datetime
import pickle
import scipy
import itertools
import statistics
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from pylab import rcParams
from time import time
from joblib import dump, load

# TENSORFLOW
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# SKLEARN
import sklearn
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# CLASSICAL ML MODELS
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


binary_labels = ['Neurosurgeon/Sr. Resident', 'Jr. Resident/Medical Student']

# MODEL INTERPRETATION
import shap

In [ ]:
# READ DATA
data = pd.read_csv('data.csv', index_col='Subject')
data = data.drop(["Stage"], axis=1)
data

In [ ]:
data.index.unique()

In [ ]:
# DETERMINING TESTING SET

skilled = data[data['Group']==0]
less_skilled = data[data['Group']==1]

skilled_nums = skilled.index.unique()
less_skilled_nums = less_skilled.index.unique()

skilled_sample = np.random.choice(skilled_nums, size=2, replace=False)
less_skilled_sample = np.random.choice(less_skilled_nums, size=3, replace=False)

testing_set = np.concatenate((skilled_sample, less_skilled_sample))
testing_set

## Hyperparameter Tuning Harness

In [ ]:
# METRICS - area under the receiver operating curve
auc = tf.keras.metrics.AUC(
    num_thresholds=200, curve='ROC', summation_method='interpolation', name=None,
    dtype=None, thresholds=None, multi_label=False, label_weights=None
)

In [ ]:
# CALLBACKS
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=30, mode='min', verbose=1)
checkpoint = ModelCheckpoint('model_best_weights.h5', monitor='val_auc', verbose=1, save_best_only=True, mode='min', save_freq=1)

In [ ]:
# WITHHOLD SOME TESTING DATA
testing_data = data[data.index.isin(testing_set)]

In [ ]:
training_data =  data[~data.index.isin(testing_set)]

In [ ]:
def build_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(50, activation='relu', input_shape=[len(train_z.keys())]))
    model.add(tf.keras.layers.Dropout(0.8))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                optimizer="adam",
                metrics=["acc"])
    return model

In [ ]:
def get_score(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    y_val_cat_prob=model.predict(x_test)
    acc = accuracy_score(y_test, y_val_cat_prob)
    return acc

In [ ]:
EPOCHS = 1000
LABEL = "Group"

# SAVE DATA AND MODEL FOR EACH OF THE LOOCV ITERATIONS
models = []
scores = []
history = []
test_predictions = []
y_trains = []
y_vals = []
x_trains = []
x_vals = []

# SAVE FINAL SCORES
scores_rf = []
scores_logistic = []
scores_svm = []
scores_nb = []
scores_lda = []
scores_knn = []


subjects = training_data.index.unique()

# LEAVE-ONE-OUT-CROSS-VALIDATION (LOOCV)
for i in range(len(subjects)):
    # CHOOSE VALIDATION SUBJECT
    val_subject = subjects[i]
    train_subjects = [subject for subject in subjects if subject != val_subject]
    print("Testing on subject #", val_subject)
    
    # MAKE TRAINING AND VALIDATION DATAFRAMES
    train_df = training_data[training_data.index.isin(train_subjects)]
    val_df = training_data[training_data.index == val_subject]
    
    # KEEP TRACK OF Y AND REMOVE IT FROM TRAINING/VALIDATION DATA
    y_trains.append(np.array(train_df[LABEL]))
    y_vals.append(np.array(val_df[LABEL]))
    train_df = train_df.drop([LABEL],axis=1)
    val_df = val_df.drop([LABEL],axis=1)
    
    # NORMALIZE
    train_z = train_df.copy(deep=True)
    val_z = val_df.copy(deep=True)
    cols = list(train_df.columns)
    for col in cols:
        train_z[col] = (train_df[col] - train_df[col].mean())/train_df[col].std(ddof=0)
        val_z[col] = (val_df[col] - train_df[col].mean())/train_df[col].std(ddof=0)
    
    # EXPORT NORMALIZED TRAINING AND VALIDATION DATA
    train_z.to_csv(str(i).zfill(2)+"_train_"+str(val_subject).zfill(2)+".csv")
    val_z.to_csv(str(i).zfill(2)+"_test_"+str(val_subject).zfill(2)+".csv")
    
    # KEEP TRACK OF TRAINING AND VALIDATION NORMALIZED DATA
    x_trains.append(train_z)
    x_vals.append(val_z)
        
    # BUILD MODEL - TUNE THIS WITH EACH HYPERPARAMETER TUNING EXPERIMENT
    models.append(build_model())
    rf = RandomForestClassifier(n_estimators = 45, max_depth=3, random_state=0)
    machine = svm.SVC(gamma='auto', kernel="rbf", decision_function_shape="ovo")
    logistic = LogisticRegression(solver='liblinear',multi_class='ovr')
    nb = GaussianNB()
    knn = KNeighborsClassifier()
    lda = LinearDiscriminantAnalysis()
    
    # EVALUATE MODELS
    history.append(models[i].fit(x_trains[i], y_trains[i],epochs=EPOCHS,callbacks = [early_stop], validation_data=(x_vals[i], y_vals[i])))
    scores.append(models[i].evaluate(x_vals[i],y_vals[i]))
    test_predictions.append(models[i].predict(x_vals[i]).flatten())
    scores_rf.append(get_score(rf, x_trains[i], x_vals[i], y_trains[i], y_vals[i]))
    scores_logistic.append(get_score(logistic, x_trains[i], x_vals[i], y_trains[i], y_vals[i]))  
    scores_svm.append(get_score(machine, x_trains[i], x_vals[i], y_trains[i], y_vals[i]))
    scores_nb.append(get_score(nb, x_trains[i], x_vals[i], y_trains[i], y_vals[i]))
    scores_lda.append(get_score(lda, x_trains[i], x_vals[i], y_trains[i], y_vals[i]))
    scores_knn.append(get_score(knn, x_trains[i], x_vals[i], y_trains[i], y_vals[i]))

In [ ]:
# GET AVERAGE MODEL SCORES
subject_scores = [score[1] for score in scores]
print("ANN:", statistics.mean(subject_scores))
print("RF:", statistics.mean(scores_rf))
print("SVM:", statistics.mean(scores_svm))
print("LR:", statistics.mean(scores_logistic))
print("NB:", statistics.mean(scores_nb))
print("KNN:", statistics.mean(scores_knn))
print("LDA:", statistics.mean(scores_lda))

In [ ]:
# IDENTIFY RESECTION MISCLASSIFICATIONS BY SUBJECT
subject_scores_dict = {}
i=0
for subject in subjects:
    subject_scores_dict[subject] = [subject_scores[i], scores_rf[i], scores_svm[i],scores_logistic[i],scores_nb[i],scores_knn[i],scores_lda[i],]
    i+=1

In [ ]:
# SAVE LOOCV RESULTS
preds = pd.pandas.DataFrame.from_dict(subject_scores_dict, orient="index", columns=["ANN", "RF", "SVM", "LR", "NB", "KNN", "LDA"])
preds.to_csv('loocv.csv')
preds

## TRAIN FINAL MODEL OF EACH TYPE

In [ ]:
def get_score(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    y_val_cat_prob=model.predict(x_test)
    fpr , tpr , thresholds = roc_curve(y_test, y_val_cat_prob)
    auroc = roc_auc_score(y_test, y_val_cat_prob)
    acc = accuracy_score(y_test, y_val_cat_prob)
    tn, fp, fn, tp = confusion_matrix(y_test, y_val_cat_prob).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    
    f = (2*precision*recall) / (precision + recall)

    return sensitivity, specificity, acc, auroc, f

In [ ]:
EPOCHS = 1000

subjects = data.index.unique()

# DROP LABEL
y_train = np.array(training_data[LABEL])
y_test = np.array(testing_data[LABEL])
train_df = training_data.drop([LABEL],axis=1)
test_df = testing_data.drop([LABEL],axis=1)
test_df.to_csv(str(i)+"_TEST_FINAL"+".csv")
train_df.to_csv(str(i)+"_TRAIN_FINAL"+".csv")

# NORMALIZE
x_train = train_df.copy(deep=True)
x_test = test_df.copy(deep=True)
columns = train_df.columns
cols = list([col for col in columns if "Stage" not in col])
for col in cols:
    x_train[col] = (train_df[col] - train_df[col].mean())/train_df[col].std(ddof=0)
    x_test[col] = (test_df[col] - train_df[col].mean())/train_df[col].std(ddof=0)

# SAVE NORMALIZED DATA
x_test.to_csv(str(i)+"_TEST_FINAL_NORMALIZED"+".csv")
x_train.to_csv(str(i)+"_TRAIN_FINAL_NORMALIZED"+".csv")    

# BUILD MODELS WITH MOST SUCCESSFUL HYPERPARAMETERS HERE
ann = build_model()
rf = RandomForestClassifier(n_estimators = 45, max_depth=3, random_state=0)
machine = svm.SVC(gamma='auto', kernel="rbf", decision_function_shape="ovo")
logistic = LogisticRegression(solver='liblinear',multi_class='ovr')
nb = GaussianNB()
knn = KNeighborsClassifier()
lda = LinearDiscriminantAnalysis()

# EVALUATE
history = ann.fit(x_train, y_train,epochs=EPOCHS,callbacks = [early_stop], validation_data=(x_test, y_test))
score_ann = ann.evaluate(x_test,y_test)
score_rf = get_score(rf, x_train, x_test, y_train, y_test)
score_logistic = get_score(logistic, x_train, x_test, y_train, y_test)
score_svm = get_score(machine, x_train, x_test, y_train, y_test)
score_nb = get_score(nb, x_train, x_test, y_train, y_test)
score_lda = get_score(lda, x_train, x_test, y_train, y_test)
score_knn = get_score(knn, x_train, x_test, y_train, y_test)

In [ ]:
# CALCULATE FULL METRICS FOR ANN
y_val_cat_prob=ann.predict(x_test)
auroc = roc_auc_score(y_test, y_val_cat_prob)
tn, fp, fn, tp = confusion_matrix(y_test, y_val_cat_prob.round()).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f = (2*precision*recall) / (precision + recall)
score_ann = [sensitivity, specificity, score_ann[1], auroc, f]

In [ ]:
# EXPORT FINAL RESULTS
results = pd.DataFrame({"ANN":score_ann,"RF":score_rf, "SVM":score_svm, "LR":score_logistic, "NB":score_nb, "KNN":score_knn, "LDA":score_lda})
results.index=["Sensitivity", "Specificity", "Accuracy", "AUROC", "F-Score"]
results = results.T
results = results.sort_values('F-Score', ascending=False)
results.to_csv("results.csv")
results

In [ ]:
# SAVE MODELS
ann.save('model_ann.h5')
dump(rf, "model_rf.joblib")
dump(machine, "model_machine.joblib")
dump(logistic, "model_logistic.joblib")
dump(nb, "model_nb.joblib")
dump(knn, "model_knn.joblib")
dump(lda, "model_lda.joblib")

## MODEL INTERPRETABILITY

In [ ]:
x = pd.concat([x_train,x_test])

In [ ]:
# use Kernel SHAP to explain test set predictions
model = ann
explainer = shap.KernelExplainer(model.predict, x)
shap_values = explainer.shap_values(x_test)

In [ ]:
shap.summary_plot(shap_values, x.columns, plot_type="bar")